<a href="https://colab.research.google.com/github/KIMMOOKYONG/COLAB-NOTENOOK/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%EC%A0%95%EB%B3%B4%EC%88%98%EC%A7%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 참조

In [ ]:
# https://bigdata-doctrine.tistory.com/3


# 변수 설정

In [ ]:
# 현재일 기준으로 코스피에 상장되어 있는 주식의 이름과 티커 수집
# 수집 기준일 설정
today = datetime.today().strftime("%Y%m%d")

# 네이버 금융 웹페이지에서 수집할 데이터의 CSS Selector 설정하기
per_selector = "#_per"
pbr_selector = "#_pbr"
eps_selector = "#_pbr"
# 배당수익률
dividend_yield_selector = "#_dvr"

# 크롤링 데이터를 저장할 리스트 변수 선언
pers = []
pbrs = []
# 배당수익률
dividend_yields = []

tickers = None
symbols = None

# 모듈 설치

In [ ]:
!pip install pykrx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 3.2 MB/s 
     |████████████████████████████████| 51 kB 281 kB/s 
     |████████████████████████████████| 251 kB 42.5 MB/s 


# 라이브러리 로딩

In [ ]:
import pandas as pd 
import numpy as np
from pykrx import stock
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# 주식 티커와 주식 이름 데이터 수집

In [ ]:
# 터커 목록 수집
# 파라미터
# date - 날짜, 문자열, 예시 20220703
# market - "KOSPI", "KOSDAQ", "KONEX", "ALL"
ticker_list = stock.get_market_ticker_list(date = today, market="KOSPI")

# 종목명 저장
symbol_list = []
# 티커에 매핑되는 종목명 수집
for ticker in ticker_list:
    symbol = stock.get_market_ticker_name(ticker)
    symbol_list.append(symbol)

In [ ]:
# 수집 데이터 갯수 확인(티커 숫자, 종목명 숫자)
len(ticker_list), len(symbol_list)

(941, 941)

# 크롤링 모듈 설치

In [ ]:
!pip install beautifulsoup4 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# PER, PBR, 배당수익률 데이터 크롤링

In [ ]:
print(ticker_list[:10])
print(symbol_list[:10])
tickers = ticker_list[:10]
symbols = symbol_list[:10]

['095570', '006840', '027410', '282330', '138930', '001460', '001465', '001040', '079160', '00104K']
['AJ네트웍스', 'AK홀딩스', 'BGF', 'BGF리테일', 'BNK금융지주', 'BYC', 'BYC우', 'CJ', 'CJ CGV', 'CJ4우(전환)']


In [ ]:
for ticker in tqdm(tickers):
    url = f"https://finance.naver.com/item/main.nhn?code={ticker}"
    html = requests.get(url, headers={'User-agent':'Mozilla/5.0'})
    soup = BeautifulSoup(html.text, "lxml")
    per = soup.select(per_selector)
    pbr = soup.select(pbr_selector)
    dividend_yield = soup.select(dividend_yield_selector)

    if not per:  # 리스트가 비어있을 경우 None으로 변환
        per = [None]
    for pe in per:
        if pe != None:
            per_text = pe.text
            per_text = per_text.replace(",", "")  # float변환을 위해 따옴표 제거
            pers.append(float(per_text))
        else:
            per_text = pe
            pers.append(per_text)

    if not pbr:  # 리스트가 비어있을 경우 None으로 변환
        pbr = [None]
    for pb in pbr:
        if pb != None:
            pbr_text = pb.text
            pbr_text = pbr_text.replace(",", "")  # float변환을 위해 따옴표 제거
            pbrs.append(float(pbr_text))
        else:
            pbr_text = pb
            pbrs.append(pbr_text)
        
    if not dividend_yield:  # 리스트가 비어있을 경우 None으로 변환
        dividend_yield = [None]
    for d in dividend_yield:
        if d != None:
            dividend_yield_text = d.text
            dividend_yield_text = dividend_yield_text.replace(",", "")  # float변환을 위해 따옴표 제거
            dividend_yields.append(float(dividend_yield_text))
        else:
            dividend_yield_text = d
            dividend_yields.append(dividend_yield_text)

100%|██████████| 10/10 [00:09<00:00,  1.03it/s]


In [ ]:
print(pers)
print(pbrs)
print(dividend_yields)

[3.65, None, 11.64, 21.14, 2.54, 12.32, 4.77, 8.95, None, 8.15]
[0.79, 0.36, 0.26, 4.26, 0.24, 0.7, 0.27, 0.5, 7.84, 0.46]
[4.19, 1.26, 2.65, 1.55, 8.21, 0.37, 0.98, 3.0, None, 3.3]


# 크롤링 데이터 데이터프레임으로 변환

In [ ]:
len(tickers), len(symbols), len(pers), len(pbrs), len(dividend_yields)

(10, 10, 10, 10, 10)

In [ ]:
df = pd.DataFrame({
    "ticker":tickers,
    "symbol":symbols,
    "per":pers,
    "pbr":pbrs,
    "dividend_yield":dividend_yields
})
df

,ticker,symbol,per,pbr,dividend_yield
0,095570,AJ네트웍스,3.65,0.79,4.19
1,006840,AK홀딩스,NaN,0.36,1.26
2,027410,BGF,11.64,0.26,2.65
3,282330,BGF리테일,21.14,4.26,1.55
4,138930,BNK금융지주,2.54,0.24,8.21
5,001460,BYC,12.32,0.70,0.37
6,001465,BYC우,4.77,0.27,0.98
7,001040,CJ,8.95,0.50,3.00
8,079160,CJ CGV,NaN,7.84,NaN
9,00104K,CJ4우(전환),8.15,0.46,3.30


In [ ]:
# per 오름차순 (NaN 제외)
per_ascending = df.sort_values(by="per", ascending=True)
per_ascending[per_ascending["per"] != None].head(20)

,ticker,symbol,per,pbr,dividend_yield
4,138930,BNK금융지주,2.54,0.24,8.21
0,095570,AJ네트웍스,3.65,0.79,4.19
6,001465,BYC우,4.77,0.27,0.98
9,00104K,CJ4우(전환),8.15,0.46,3.30
7,001040,CJ,8.95,0.50,3.00
2,027410,BGF,11.64,0.26,2.65
5,001460,BYC,12.32,0.70,0.37
3,282330,BGF리테일,21.14,4.26,1.55
1,006840,AK홀딩스,NaN,0.36,1.26
8,079160,CJ CGV,NaN,7.84,NaN


In [ ]:
# pbr 오름차순 (NaN 제외)
pbr_ascending = df.sort_values(by="pbr", ascending=True)
pbr_ascending[pbr_ascending["pbr"] != None].head(20)

,ticker,symbol,per,pbr,dividend_yield
4,138930,BNK금융지주,2.54,0.24,8.21
2,027410,BGF,11.64,0.26,2.65
6,001465,BYC우,4.77,0.27,0.98
1,006840,AK홀딩스,NaN,0.36,1.26
9,00104K,CJ4우(전환),8.15,0.46,3.30
7,001040,CJ,8.95,0.50,3.00
5,001460,BYC,12.32,0.70,0.37
0,095570,AJ네트웍스,3.65,0.79,4.19
3,282330,BGF리테일,21.14,4.26,1.55
8,079160,CJ CGV,NaN,7.84,NaN


In [ ]:
# 배당수익률 내림차순
df.sort_values(by="dividend_yield", ascending=False).head(20)

,ticker,symbol,per,pbr,dividend_yield
4,138930,BNK금융지주,2.54,0.24,8.21
0,095570,AJ네트웍스,3.65,0.79,4.19
9,00104K,CJ4우(전환),8.15,0.46,3.30
7,001040,CJ,8.95,0.50,3.00
2,027410,BGF,11.64,0.26,2.65
3,282330,BGF리테일,21.14,4.26,1.55
1,006840,AK홀딩스,NaN,0.36,1.26
6,001465,BYC우,4.77,0.27,0.98
5,001460,BYC,12.32,0.70,0.37
8,079160,CJ CGV,NaN,7.84,NaN


In [ ]:
# csv로 저장
df.to_csv(f"{today}_df.csv")

In [ ]:
# 개별 종목별 EPS 수집방법
# https://scribblinganything.tistory.com/383